# Uppmärksamhetsmekanismer och transformatorer

En stor nackdel med rekurrenta nätverk är att alla ord i en sekvens har samma påverkan på resultatet. Detta leder till suboptimal prestanda med standard LSTM encoder-decoder-modeller för sekvens-till-sekvens-uppgifter, såsom namngiven entity-igenkänning och maskinöversättning. I verkligheten har specifika ord i inmatningssekvensen ofta större påverkan på sekventiella utgångar än andra.

Tänk på en sekvens-till-sekvens-modell, såsom maskinöversättning. Den implementeras med två rekurrenta nätverk, där ett nätverk (**encoder**) komprimerar inmatningssekvensen till ett dolt tillstånd, och ett annat, **decoder**, vecklar ut detta dolda tillstånd till det översatta resultatet. Problemet med denna metod är att det slutliga tillståndet i nätverket har svårt att komma ihåg början av en mening, vilket leder till dålig modellkvalitet för långa meningar.

**Uppmärksamhetsmekanismer** ger ett sätt att vikta den kontextuella påverkan av varje inmatningsvektor på varje utgångsprediktion av RNN. Detta implementeras genom att skapa genvägar mellan mellanliggande tillstånd i inmatnings-RNN och utgångs-RNN. På detta sätt, när vi genererar utgångssymbolen $y_t$, tar vi hänsyn till alla dolda tillstånd $h_i$ från inmatningen, med olika viktkoefficienter $\alpha_{t,i}$. 

![Bild som visar en encoder/decoder-modell med ett additivt uppmärksamhetslager](../../../../../translated_images/sv/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder-modellen med additiv uppmärksamhetsmekanism i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citerad från [denna bloggpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Uppmärksamhetsmatrisen $\{\alpha_{i,j}\}$ representerar graden av påverkan som vissa inmatningsord har vid genereringen av ett visst ord i utgångssekvensen. Nedan är ett exempel på en sådan matris:

![Bild som visar ett exempel på justering funnen av RNNsearch-50, hämtad från Bahdanau - arviz.org](../../../../../translated_images/sv/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figur hämtad från [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Uppmärksamhetsmekanismer är ansvariga för mycket av den nuvarande eller nära nuvarande topprestandan inom naturlig språkbehandling. Att lägga till uppmärksamhet ökar dock kraftigt antalet modellparametrar, vilket ledde till skalningsproblem med RNNs. En viktig begränsning för att skala RNNs är att den rekurrenta naturen hos modellerna gör det utmanande att batcha och parallellisera träning. I en RNN måste varje element i en sekvens bearbetas i sekventiell ordning, vilket innebär att det inte enkelt kan parallelliseras.

Adoptionen av uppmärksamhetsmekanismer i kombination med denna begränsning ledde till skapandet av de nuvarande toppmodellerna, transformatorer, som vi idag känner och använder, från BERT till OpenGPT3.

## Transformator-modeller

Istället för att vidarebefordra kontexten från varje tidigare prediktion till nästa utvärderingssteg, använder **transformator-modeller** **positionella kodningar** och **uppmärksamhet** för att fånga kontexten av en given inmatning inom ett angivet textfönster. Bilden nedan visar hur positionella kodningar med uppmärksamhet kan fånga kontext inom ett givet fönster.

![Animerad GIF som visar hur utvärderingar utförs i transformator-modeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Eftersom varje inmatningsposition mappas oberoende till varje utgångsposition, kan transformatorer parallellisera bättre än RNNs, vilket möjliggör mycket större och mer uttrycksfulla språkmodeller. Varje uppmärksamhetshuvud kan användas för att lära sig olika relationer mellan ord, vilket förbättrar nedströmsuppgifter inom naturlig språkbehandling.

## Bygga en enkel transformator-modell

Keras innehåller inte en inbyggd transformatorlager, men vi kan bygga vår egen. Som tidigare kommer vi att fokusera på textklassificering av AG News-datasetet, men det är värt att nämna att transformator-modeller visar bästa resultat vid mer komplexa NLP-uppgifter.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nya lager i Keras bör ärva `Layer`-klassen och implementera metoden `call`. Låt oss börja med **Positional Embedding**-lagret. Vi kommer att använda [några kodexempel från den officiella Keras-dokumentationen](https://keras.io/examples/nlp/text_classification_with_transformer/). Vi antar att vi fyller alla inmatningssekvenser till längden `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Detta lager består av två `Embedding`-lager: ett för att skapa inbäddningar av tokens (på det sätt vi tidigare har diskuterat) och ett för att skapa inbäddningar av tokenpositioner. Tokenpositionerna skapas som en sekvens av naturliga tal från 0 till `maxlen` med hjälp av `tf.range`, och skickas sedan genom inbäddningslagret. De två resulterande inbäddningsvektorerna adderas, vilket ger en positionsinbäddad representation av indata med formen `maxlen`$\times$`embed_dim`.

Nu ska vi implementera transformerblocket. Det kommer att ta utdata från det tidigare definierade inbäddningslagret:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformern tillämpar `MultiHeadAttention` på den positionskodade inmatningen för att generera uppmärksamhetsvektorn med dimensionen `maxlen`$\times$`embed_dim`, som sedan blandas med inmatningen och normaliseras med hjälp av `LayerNormalization`.

> **Obs**: `LayerNormalization` liknar `BatchNormalization` som diskuterades i *Datorseende*-delen av denna utbildningsväg, men den normaliserar utgångarna från det föregående lagret för varje träningsprov oberoende, för att få dem inom intervallet [-1..1].

Utgången från detta lager skickas sedan genom ett `Dense`-nätverk (i vårt fall - ett tvålagers perceptron), och resultatet adderas till den slutliga utgången (som normaliseras igen).

Nu är vi redo att definiera den kompletta transformer-modellen:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer-modeller

**BERT** (Bidirectional Encoder Representations from Transformers) är ett mycket stort flerskiktat transformatornätverk med 12 lager för *BERT-base* och 24 för *BERT-large*. Modellen förtränas först på en stor mängd textdata (WikiPedia + böcker) med hjälp av osuperviserad träning (förutsäga maskerade ord i en mening). Under förträningen absorberar modellen en betydande nivå av språkförståelse som sedan kan utnyttjas med andra dataset genom finjustering. Denna process kallas **transfer learning**.

![bild från http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sv/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Det finns många variationer av Transformer-arkitekturer, inklusive BERT, DistilBERT, BigBird, OpenGPT3 och fler som kan finjusteras.

Låt oss se hur vi kan använda en förtränad BERT-modell för att lösa vårt traditionella sekvensklassificeringsproblem. Vi kommer att låna idéer och viss kod från [officiell dokumentation](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

För att ladda förtränade modeller kommer vi att använda **Tensorflow hub**. Först, låt oss ladda den BERT-specifika vektoriseringen:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Det är viktigt att du använder samma vektorisering som det ursprungliga nätverket tränades med. Dessutom returnerar BERT-vektoriseraren tre komponenter:
* `input_word_ids`, som är en sekvens av tokennummer för inmatningsmeningen
* `input_mask`, som visar vilken del av sekvensen som innehåller faktisk inmatning och vilken del som är utfyllnad. Detta liknar masken som produceras av `Masking`-lagret
* `input_type_ids` används för språkmodellering och gör det möjligt att specificera två inmatningsmeningar i en sekvens.

Därefter kan vi instansiera BERT-funktionsutdragaren:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Så, BERT-lagret returnerar ett antal användbara resultat:
* `pooled_output` är resultatet av att genomsnittligt beräkna alla tokens i sekvensen. Du kan se det som en intelligent semantisk inbäddning av hela nätverket. Det motsvarar utdata från lagret `GlobalAveragePooling1D` i vår tidigare modell.
* `sequence_output` är utdatat från det sista transformerlagret (motsvarar utdatat från `TransformerBlock` i vår modell ovan).
* `encoder_outputs` är utdatat från alla transformerlager. Eftersom vi har laddat en 4-lagers BERT-modell (som du förmodligen kan gissa från namnet, som innehåller `4_H`), har den 4 tensorer. Den sista är densamma som `sequence_output`.

Nu ska vi definiera den end-to-end klassificeringsmodellen. Vi kommer att använda *funktionell modelldefinition*, där vi definierar modellens indata och sedan tillhandahåller en serie uttryck för att beräkna dess utdata. Vi kommer också att göra BERT-modellens vikter icke-träningsbara och endast träna den slutliga klassificeraren:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Trots att det finns få träningsbara parametrar är processen ganska långsam, eftersom BERT-funktionsutvinnaren är beräkningsmässigt tung. Det verkar som att vi inte kunde uppnå rimlig noggrannhet, antingen på grund av brist på träning eller brist på modellparametrar.

Låt oss försöka låsa upp BERT-vikterna och träna den också. Detta kräver en mycket låg inlärningshastighet och en mer noggrann träningsstrategi med **uppvärmning**, med hjälp av **AdamW**-optimeraren. Vi kommer att använda paketet `tf-models-official` för att skapa optimeraren:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Som du kan se går träningen ganska långsamt - men du kanske vill experimentera och träna modellen i några få epoker (5-10) och se om du kan få det bästa resultatet jämfört med de metoder vi har använt tidigare.

## Huggingface Transformers-biblioteket

Ett annat mycket vanligt (och lite enklare) sätt att använda Transformer-modeller är [HuggingFace-paketet](https://github.com/huggingface/), som tillhandahåller enkla byggstenar för olika NLP-uppgifter. Det är tillgängligt både för Tensorflow och PyTorch, ett annat mycket populärt ramverk för neurala nätverk.

> **Note**: Om du inte är intresserad av att se hur Transformers-biblioteket fungerar - kan du hoppa till slutet av denna notebook, eftersom du inte kommer att se något väsentligt annorlunda från vad vi har gjort ovan. Vi kommer att upprepa samma steg för att träna BERT-modellen med ett annat bibliotek och en betydligt större modell. Processen innebär därför ganska lång träning, så du kanske bara vill titta igenom koden.

Låt oss se hur vårt problem kan lösas med [Huggingface Transformers](http://huggingface.co).


Det första vi behöver göra är att välja den modell som vi ska använda. Förutom några inbyggda modeller innehåller Huggingface ett [online-modellarkiv](https://huggingface.co/models), där du kan hitta många fler förtränade modeller från communityn. Alla dessa modeller kan laddas och användas bara genom att ange ett modellnamn. Alla nödvändiga binärfiler för modellen laddas automatiskt ner.

I vissa fall kan det vara nödvändigt att ladda dina egna modeller, och då kan du specificera katalogen som innehåller alla relevanta filer, inklusive parametrar för tokenizer, `config.json`-filen med modellparametrar, binära vikter, etc.

Utifrån modellnamnet kan vi instansiera både modellen och tokenizern. Låt oss börja med en tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objektet innehåller `encode`-funktionen som kan användas direkt för att koda text:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Vi kan också använda tokenizer för att koda en sekvens på ett sätt som är lämpligt för att skicka till modellen, dvs. inklusive `token_ids`, `input_mask`-fält, etc. Vi kan också ange att vi vill ha Tensorflow-tensorer genom att tillhandahålla argumentet `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

I vårt fall kommer vi att använda den förtränade BERT-modellen som heter `bert-base-uncased`. *Uncased* betyder att modellen är skiftlägesokänslig.

När vi tränar modellen behöver vi ge en tokeniserad sekvens som input, och därför kommer vi att utforma en databehandlingspipeline. Eftersom `tokenizer.encode` är en Python-funktion, kommer vi att använda samma metod som i den senaste enheten genom att kalla den med `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nu kan vi ladda den faktiska modellen med hjälp av paketet `BertForSequenceClassification`. Detta säkerställer att vår modell redan har en nödvändig arkitektur för klassificering, inklusive den slutliga klassificeraren. Du kommer att se ett varningsmeddelande som anger att vikterna för den slutliga klassificeraren inte är initialiserade, och att modellen skulle kräva förträning - det är helt i sin ordning, eftersom det är precis vad vi är på väg att göra!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Som du kan se från `summary()`, innehåller modellen nästan 110 miljoner parametrar! Förmodligen, om vi vill ha en enkel klassificeringsuppgift på en relativt liten dataset, vill vi inte träna BERT-baslagret:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nu är vi redo att börja träna!

> **Note**: Att träna en fullskalig BERT-modell kan vara väldigt tidskrävande! Därför kommer vi bara att träna den för de första 32 batcherna. Detta är bara för att visa hur modellträning är uppsatt. Om du är intresserad av att prova fullskalig träning - ta bara bort parametrarna `steps_per_epoch` och `validation_steps`, och förbered dig på att vänta!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Om du ökar antalet iterationer och väntar tillräckligt länge, och tränar i flera epoker, kan du förvänta dig att BERT-klassificering ger oss den bästa noggrannheten! Det beror på att BERT redan förstår språkets struktur ganska väl, och vi behöver bara finjustera den slutliga klassificeraren. Men eftersom BERT är en stor modell tar hela träningsprocessen lång tid och kräver rejäl beräkningskraft! (GPU, och helst fler än en).

> **Note:** I vårt exempel har vi använt en av de minsta förtränade BERT-modellerna. Det finns större modeller som sannolikt ger bättre resultat.


## Sammanfattning

I den här enheten har vi tittat på mycket aktuella modellarkitekturer baserade på **transformers**. Vi har använt dem för vår textklassificeringsuppgift, men på samma sätt kan BERT-modeller användas för entity extraction, fråge-svar och andra NLP-uppgifter.

Transformer-modeller representerar det nuvarande toppskiktet inom NLP, och i de flesta fall bör det vara den första lösningen du börjar experimentera med när du implementerar skräddarsydda NLP-lösningar. Dock är det mycket viktigt att förstå de grundläggande principerna bakom rekurrenta neurala nätverk som diskuterats i denna modul om du vill bygga avancerade neurala modeller.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
